# Nonparametric ML Models - Cumulative Lab

## Introduction

In this cumulative lab, you will apply two nonparametric models you have just learned — k-nearest neighbors and decision trees — to the forest cover dataset.

## Objectives

* Practice identifying and applying appropriate preprocessing steps
* Perform an iterative modeling process, starting from a baseline model
* Explore multiple model algorithms, and tune their hyperparameters
* Practice choosing a final model across multiple model algorithms and evaluating its performance

## Your Task: Complete an End-to-End ML Process with Nonparametric Models on the Forest Cover Dataset

![line of pine trees](https://curriculum-content.s3.amazonaws.com/data-science/images/trees.jpg)

Photo by <a href="https://unsplash.com/@michaelbenz?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Michael Benz</a> on <a href="/s/photos/forest?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>

### Business and Data Understanding

To repeat the previous description:

> Here we will be using an adapted version of the forest cover dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/covertype). Each record represents a 30 x 30 meter cell of land within Roosevelt National Forest in northern Colorado, which has been labeled as `Cover_Type` 1 for "Cottonwood/Willow" and `Cover_Type` 0 for "Ponderosa Pine". (The original dataset contained 7 cover types but we have simplified it.)

The task is to predict the `Cover_Type` based on the available cartographic variables:

In [1]:
# Run this cell without changes
import pandas as pd

df = pd.read_csv('data/forest_cover.csv')
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2553,235,17,351,95,780,188,253,199,1410,...,0,0,0,0,0,0,0,0,0,0
1,2011,344,17,313,29,404,183,211,164,300,...,0,0,0,0,0,0,0,0,0,0
2,2022,24,13,391,42,509,212,212,134,421,...,0,0,0,0,0,0,0,0,0,0
3,2038,50,17,408,71,474,226,200,102,283,...,0,0,0,0,0,0,0,0,0,0
4,2018,341,27,351,34,390,152,188,168,190,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,0
38497,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,0
38498,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,0
38499,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,0


> As you can see, we have over 38,000 rows, each with 52 feature columns and 1 target column:

> * `Elevation`: Elevation in meters
> * `Aspect`: Aspect in degrees azimuth
> * `Slope`: Slope in degrees
> * `Horizontal_Distance_To_Hydrology`: Horizontal dist to nearest surface water features in meters
> * `Vertical_Distance_To_Hydrology`: Vertical dist to nearest surface water features in meters
> * `Horizontal_Distance_To_Roadways`: Horizontal dist to nearest roadway in meters
> * `Hillshade_9am`: Hillshade index at 9am, summer solstice
> * `Hillshade_Noon`: Hillshade index at noon, summer solstice
> * `Hillshade_3pm`: Hillshade index at 3pm, summer solstice
> * `Horizontal_Distance_To_Fire_Points`: Horizontal dist to nearest wildfire ignition points, meters
> * `Wilderness_Area_x`: Wilderness area designation (3 columns)
> * `Soil_Type_x`: Soil Type designation (39 columns)
> * `Cover_Type`: 1 for cottonwood/willow, 0 for ponderosa pine

This is also an imbalanced dataset, since cottonwood/willow trees are relatively rare in this forest:

In [2]:
# Run this cell without changes
print("Raw Counts")
print(df["Cover_Type"].value_counts())
print()
print("Percentages")
print(df["Cover_Type"].value_counts(normalize=True))

Raw Counts
Cover_Type
0    35754
1     2747
Name: count, dtype: int64

Percentages
Cover_Type
0    0.928651
1    0.071349
Name: proportion, dtype: float64


Thus, a baseline model that always chose the majority class would have an accuracy of over 92%. Therefore we will want to report additional metrics at the end.

### Previous Best Model

In a previous lab, we used SMOTE to create additional synthetic data, then tuned the hyperparameters of a logistic regression model to get the following final model metrics:

* **Log loss:** 0.13031294393913376
* **Accuracy:** 0.9456679825472678
* **Precision:** 0.6659919028340081
* **Recall:** 0.47889374090247455

In this lab, you will try to beat those scores using more-complex, nonparametric models.

### Modeling

Although you may be aware of some additional model algorithms available from scikit-learn, for this lab you will be focusing on two of them: k-nearest neighbors and decision trees. Here are some reminders about these models:

#### kNN - [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

This algorithm — unlike linear models or tree-based models — does not emphasize learning the relationship between the features and the target. Instead, for a given test record, it finds the most similar records in the training set and returns an average of their target values.

* **Training speed:** Fast. In theory it's just saving the training data for later, although the scikit-learn implementation has some additional logic "under the hood" to make prediction faster.
* **Prediction speed:** Very slow. The model has to look at every record in the training set to find the k closest to the new record.
* **Requires scaling:** Yes. The algorithm to find the nearest records is distance-based, so it matters that distances are all on the same scale.
* **Key hyperparameters:** `n_neighbors` (how many nearest neighbors to find; too few neighbors leads to overfitting, too many leads to underfitting), `p` and `metric` (what kind of distance to use in defining "nearest" neighbors)

#### Decision Trees - [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

Similar to linear models (and unlike kNN), this algorithm emphasizes learning the relationship between the features and the target. However, unlike a linear model that tries to find linear relationships between each of the features and the target, decision trees look for ways to split the data based on features to decrease the entropy of the target in each split.

* **Training speed:** Slow. The model is considering splits based on as many as all of the available features, and it can split on the same feature multiple times. This requires exponential computational time that increases based on the number of columns as well as the number of rows.
* **Prediction speed:** Medium fast. Producing a prediction with a decision tree means applying several conditional statements, which is slower than something like logistic regression but faster than kNN.
* **Requires scaling:** No. This model is not distance-based. You also can use a `LabelEncoder` rather than `OneHotEncoder` for categorical data, since this algorithm doesn't necessarily assume that the distance between `1` and `2` is the same as the distance between `2` and `3`.
* **Key hyperparameters:** Many features relating to "pruning" the tree. By default they are set so the tree can overfit, and by setting them higher or lower (depending on the hyperparameter) you can reduce overfitting, but too much will lead to underfitting. These are: `max_depth`, `min_samples_split`, `min_samples_leaf`, `min_weight_fraction_leaf`, `max_features`, `max_leaf_nodes`, and `min_impurity_decrease`. You can also try changing the `criterion` to "entropy" or the `splitter` to "random" if you want to change the splitting logic.

### Requirements

#### 1. Prepare the Data for Modeling

#### 2. Build a Baseline kNN Model

#### 3. Build Iterative Models to Find the Best kNN Model

#### 4. Build a Baseline Decision Tree Model

#### 5. Build Iterative Models to Find the Best Decision Tree Model

#### 6. Choose and Evaluate an Overall Best Model

## 1. Prepare the Data for Modeling

The target is `Cover_Type`. In the cell below, split `df` into `X` and `y`, then perform a train-test split with `random_state=42` and `stratify=y` to create variables with the standard `X_train`, `X_test`, `y_train`, `y_test` names.

Include the relevant imports as you go.

In [3]:
# Your code here
X = df.drop('Cover_Type', axis=1)
y = df['Cover_Type']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

Now, instantiate a `StandardScaler`, fit it on `X_train`, and create new variables `X_train_scaled` and `X_test_scaled` containing values transformed with the scaler.

In [4]:
# Your code here
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames to preserve column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

The following code checks that everything is set up correctly:

In [5]:
# Run this cell without changes

# Checking that df was separated into correct X and y
assert type(X) == pd.DataFrame and X.shape == (38501, 52)
assert type(y) == pd.Series and y.shape == (38501,)

# Checking the train-test split
assert type(X_train) == pd.DataFrame and X_train.shape == (28875, 52)
assert type(X_test) == pd.DataFrame and X_test.shape == (9626, 52)
assert type(y_train) == pd.Series and y_train.shape == (28875,)
assert type(y_test) == pd.Series and y_test.shape == (9626,)

# Checking the scaling
assert X_train_scaled.shape == X_train.shape
assert round(X_train_scaled[0][0], 3) == -0.636
assert X_test_scaled.shape == X_test.shape
assert round(X_test_scaled[0][0], 3) == -1.370

KeyError: 0

## 2. Build a Baseline kNN Model

Build a scikit-learn kNN model with default hyperparameters. Then use `cross_val_score` with `scoring="neg_log_loss"` to find the mean log loss for this model (passing in `X_train_scaled` and `y_train` to `cross_val_score`). You'll need to find the mean of the cross-validated scores, and negate the value (either put a `-` at the beginning or multiply by `-1`) so that your answer is a log loss rather than a negative log loss.

Call the resulting score `knn_baseline_log_loss`.

Your code might take a minute or more to run.

In [7]:
# Replace None with appropriate code
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Creating the model
knn_baseline_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)
# Perform cross-validation
knn_baseline_log_loss = cross_val_score(knn_baseline_model, X_train, y_train, scoring= 'neg_log_loss')

knn_baseline_log_loss

array([-0.07731441, -0.07450023, -0.06439287, -0.08224353, -0.07849675])

Our best logistic regression model had a log loss of 0.13031294393913376

Is this model better? Compare it in terms of metrics and speed.

In [8]:
# Replace None with appropriate text
"""
KNN is SIGNIFICANTLY BETTER in terms of metrics:

KNN Log Loss: ~0.075 (from your CV scores: -0.077, -0.074, -0.064, -0.082, -0.078)

Logistic Regression Log Loss: 0.130313

Improvement: ~42% lower log loss with KNN

⚡ Speed Considerations:

KNN: Fast training (just stores data), but slow prediction (computes distances to all training points)

Logistic Regression: Slower training (optimization), but very fast prediction
"""

'\nKNN is SIGNIFICANTLY BETTER in terms of metrics:\n\nKNN Log Loss: ~0.075 (from your CV scores: -0.077, -0.074, -0.064, -0.082, -0.078)\n\nLogistic Regression Log Loss: 0.130313\n\nImprovement: ~42% lower log loss with KNN\n\n⚡ Speed Considerations:\n\nKNN: Fast training (just stores data), but slow prediction (computes distances to all training points)\n\nLogistic Regression: Slower training (optimization), but very fast prediction\n'

## 3. Build Iterative Models to Find the Best kNN Model

Build and evaluate at least two more kNN models to find the best one. Explain why you are changing the hyperparameters you are changing as you go. These models will be *slow* to run, so be thinking about what you might try next as you run them.

In [9]:
# Your code here (add more cells as needed)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import time

# Model 2: Find optimal k with distance weighting
print("=== MODEL 2: Finding Optimal k ===")

# Test different k values
k_values = [3, 5, 7, 9, 11, 15, 21]
results = []

for k in k_values:
    start_time = time.time()
    knn_model = KNeighborsClassifier(
        n_neighbors=k,
        weights='distance',  # Closer neighbors have more influence
        algorithm='auto',
        n_jobs=-1  # Use all cores
    )
    
    cv_scores = cross_val_score(knn_model, X_train_scaled, y_train, scoring='neg_log_loss', cv=3)  # Reduced CV for speed
    mean_score = -cv_scores.mean()
    training_time = time.time() - start_time
    
    results.append({
        'k': k,
        'log_loss': mean_score,
        'time': training_time
    })
    
    print(f"k={k}: Log Loss = {mean_score:.6f}, Time = {training_time:.2f}s")

# Find best k
best_k_model2 = min(results, key=lambda x: x['log_loss'])
print(f"\n🎯 BEST k: {best_k_model2['k']} with Log Loss = {best_k_model2['log_loss']:.6f}")

=== MODEL 2: Finding Optimal k ===
k=3: Log Loss = 0.212854, Time = 12.10s
k=5: Log Loss = 0.134808, Time = 4.21s
k=7: Log Loss = 0.105102, Time = 4.10s
k=9: Log Loss = 0.086797, Time = 4.01s
k=11: Log Loss = 0.079743, Time = 4.29s
k=15: Log Loss = 0.070903, Time = 4.14s
k=21: Log Loss = 0.068884, Time = 4.07s

🎯 BEST k: 21 with Log Loss = 0.068884


In [10]:
# Your code here (add more cells as needed)
# Model 3: Advanced parameter combinations
print("\n=== MODEL 3: Advanced Parameter Tuning ===")

# Test different combinations
param_combinations = [
    {'n_neighbors': best_k_model2['k'], 'weights': 'distance', 'p': 1, 'metric': 'manhattan'},  # Manhattan distance
    {'n_neighbors': best_k_model2['k'], 'weights': 'distance', 'p': 2, 'metric': 'minkowski'},  # Euclidean (default)
    {'n_neighbors': best_k_model2['k'], 'weights': 'uniform', 'p': 2, 'metric': 'minkowski'},   # Uniform weights
    {'n_neighbors': max(3, best_k_model2['k'] - 2), 'weights': 'distance', 'p': 2, 'metric': 'minkowski'},  # Smaller k
]

advanced_results = []

for i, params in enumerate(param_combinations):
    start_time = time.time()
    
    knn_model = KNeighborsClassifier(
        n_neighbors=params['n_neighbors'],
        weights=params['weights'],
        p=params['p'],
        metric=params['metric'],
        n_jobs=-1
    )
    
    cv_scores = cross_val_score(knn_model, X_train_scaled, y_train, scoring='neg_log_loss', cv=3)
    mean_score = -cv_scores.mean()
    training_time = time.time() - start_time
    
    advanced_results.append({
        'params': params,
        'log_loss': mean_score,
        'time': training_time
    })
    
    print(f"Config {i+1}: k={params['n_neighbors']}, weights={params['weights']}, p={params['p']}")
    print(f"  Log Loss = {mean_score:.6f}, Time = {training_time:.2f}s")

# Find best advanced model
best_advanced = min(advanced_results, key=lambda x: x['log_loss'])
print(f"\n🎯 BEST ADVANCED: k={best_advanced['params']['n_neighbors']}, "
      f"weights={best_advanced['params']['weights']}, p={best_advanced['params']['p']}")
print(f"Log Loss = {best_advanced['log_loss']:.6f}")


=== MODEL 3: Advanced Parameter Tuning ===
Config 1: k=21, weights=distance, p=1
  Log Loss = 0.066245, Time = 21.69s
Config 2: k=21, weights=distance, p=2
  Log Loss = 0.068884, Time = 4.42s
Config 3: k=21, weights=uniform, p=2
  Log Loss = 0.076654, Time = 4.43s
Config 4: k=19, weights=distance, p=2
  Log Loss = 0.068385, Time = 4.50s

🎯 BEST ADVANCED: k=21, weights=distance, p=1
Log Loss = 0.066245


In [13]:
# Your code here (add more cells as needed)
# Model 4: Final optimized model with best parameters
print("\n=== MODEL 4: Final Optimized Model ===")

# Use best parameters from previous experiments
final_knn = KNeighborsClassifier(
    n_neighbors=best_advanced['params']['n_neighbors'],
    weights=best_advanced['params']['weights'],
    p=best_advanced['params']['p'],
    metric=best_advanced['params']['metric'],
    algorithm='kd_tree',  # Often faster for moderate dimensions
    n_jobs=-1
)

# Final evaluation with proper 5-fold CV
start_time = time.time()
final_cv_scores = cross_val_score(final_knn, X_train_scaled, y_train, scoring='neg_log_loss', cv=5)
final_time = time.time() - start_time

final_log_loss = -final_cv_scores.mean()
final_std = final_cv_scores.std()

print(f"FINAL MODEL PERFORMANCE:")
print(f"Parameters: k={best_advanced['params']['n_neighbors']}, "
      f"weights={best_advanced['params']['weights']}, p={best_advanced['params']['p']}")
print(f"Log Loss: {final_log_loss:.6f} ± {final_std:.6f}")
print(f"CV Scores: {-final_cv_scores}")
print(f"Training Time: {final_time:.2f}s")

# Compare all models
print(f"\n=== COMPARISON SUMMARY ===")
print(f"Best k from Model 2:            {best_k_model2['log_loss']:.6f}")
print(f"Best advanced from Model 3:     {best_advanced['log_loss']:.6f}")
print(f"Final optimized Model 4:        {final_log_loss:.6f}")
print(f"Original Logistic Regression:   0.130313")

improvement = (0.130313 - final_log_loss) / 0.130313 * 100
print(f"\nOverall improvement over Logistic Regression: {improvement:.1f}%")


=== MODEL 4: Final Optimized Model ===
FINAL MODEL PERFORMANCE:
Parameters: k=21, weights=distance, p=1
Log Loss: 0.055427 ± 0.003368
CV Scores: [0.05078115 0.0609295  0.05633983 0.05565002 0.05343233]
Training Time: 34.40s

=== COMPARISON SUMMARY ===
Best k from Model 2:            0.068884
Best advanced from Model 3:     0.066245
Final optimized Model 4:        0.055427
Original Logistic Regression:   0.130313

Overall improvement over Logistic Regression: 57.5%


## 4. Build a Baseline Decision Tree Model

Now that you have chosen your best kNN model, start investigating decision tree models. First, build and evaluate a baseline decision tree model, using default hyperparameters (with the exception of `random_state=42` for reproducibility).

(Use cross-validated log loss, just like with the previous models.)

In [14]:
# Your code here
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Build baseline decision tree
dt_baseline = DecisionTreeClassifier(random_state=42)

# Perform cross-validation
dt_baseline_scores = cross_val_score(
    dt_baseline, 
    X_train_scaled, 
    y_train, 
    scoring='neg_log_loss', 
    cv=5
)

# Calculate mean log loss
dt_baseline_log_loss = -dt_baseline_scores.mean()

print(f"Baseline Decision Tree Log Loss: {dt_baseline_log_loss:.6f}")
print(f"CV Scores: {-dt_baseline_scores}")

Baseline Decision Tree Log Loss: 0.736476
CV Scores: [0.6178912  0.74895903 0.6927871  0.81137228 0.81137228]


Interpret this score. How does this compare to the log loss from our best logistic regression and best kNN models? Any guesses about why?

In [ ]:
# Replace None with appropriate text
"""
None
"""

## 5. Build Iterative Models to Find the Best Decision Tree Model

Build and evaluate at least two more decision tree models to find the best one. Explain why you are changing the hyperparameters you are changing as you go.

In [15]:
# Your code here (add more cells as needed)
# Model 1: Baseline (Already done)
print("Model 1 - Baseline:")
print(f"Log Loss: {dt_baseline_log_loss:.6f}")
print("Parameters: Default (unconstrained tree)\n")

Model 1 - Baseline:
Log Loss: 0.736476
Parameters: Default (unconstrained tree)



In [16]:
# Your code here (add more cells as needed)
# Model 2: Regularized to prevent overfitting
print("=== Model 2: Regularized Tree ===")
dt_reg = DecisionTreeClassifier(
    max_depth=10,           # Limit tree depth to prevent overfitting
    min_samples_split=20,   # Require more samples to split
    min_samples_leaf=10,    # Require more samples per leaf
    random_state=42
)

dt_reg_scores = cross_val_score(dt_reg, X_train_scaled, y_train, scoring='neg_log_loss', cv=5)
dt_reg_log_loss = -dt_reg_scores.mean()

print(f"Log Loss: {dt_reg_log_loss:.6f}")
print("Parameters: max_depth=10, min_samples_split=20, min_samples_leaf=10")
print("Reason: Prevent overfitting by constraining tree growth\n")

=== Model 2: Regularized Tree ===
Log Loss: 0.204135
Parameters: max_depth=10, min_samples_split=20, min_samples_leaf=10
Reason: Prevent overfitting by constraining tree growth



In [17]:
# Your code here (add more cells as needed)
# Model 3: Balanced class weights + different criterion
print("=== Model 3: Balanced Tree ===")
dt_bal = DecisionTreeClassifier(
    max_depth=15,
    min_samples_split=15,
    criterion='entropy',    # Try entropy instead of gini
    class_weight='balanced', # Handle class imbalance
    random_state=42
)

dt_bal_scores = cross_val_score(dt_bal, X_train_scaled, y_train, scoring='neg_log_loss', cv=5)
dt_bal_log_loss = -dt_bal_scores.mean()

print(f"Log Loss: {dt_bal_log_loss:.6f}")
print("Parameters: max_depth=15, min_samples_split=15, criterion='entropy', class_weight='balanced'")
print("Reason: Handle class imbalance and try different split criterion")

# Comparison
print(f"\n=== COMPARISON ===")
print(f"Baseline:    {dt_baseline_log_loss:.6f}")
print(f"Regularized: {dt_reg_log_loss:.6f}")
print(f"Balanced:    {dt_bal_log_loss:.6f}")

=== Model 3: Balanced Tree ===
Log Loss: 0.421253
Parameters: max_depth=15, min_samples_split=15, criterion='entropy', class_weight='balanced'
Reason: Handle class imbalance and try different split criterion

=== COMPARISON ===
Baseline:    0.736476
Regularized: 0.204135
Balanced:    0.421253


## 6. Choose and Evaluate an Overall Best Model

Which model had the best performance? What type of model was it?

Instantiate a variable `final_model` using your best model with the best hyperparameters.

In [18]:
# Replace None with appropriate code
final_model = KNeighborsClassifier(
    n_neighbors=best_advanced['params']['n_neighbors'],
    weights=best_advanced['params']['weights'],
    p=best_advanced['params']['p'],
    metric=best_advanced['params']['metric'],
    algorithm='kd_tree',  # Often faster for moderate dimensions
    n_jobs=-1
)

# Fit the model on the full training data
# (scaled or unscaled depending on the model)
final_model.fit(X_train_scaled, y_train)

,n_neighbors,21
,weights,'distance'
,algorithm,'kd_tree'
,leaf_size,30
,p,1
,metric,'manhattan'
,metric_params,None
,n_jobs,-1


Now, evaluate the log loss, accuracy, precision, and recall. This code is mostly filled in for you, but you need to replace `None` with either `X_test` or `X_test_scaled` depending on the model you chose.

In [20]:
# Replace None with appropriate code
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss

preds = final_model.predict(X_test)
probs = final_model.predict_proba(X_test_scaled)

print("log loss: ", log_loss(y_test, probs))
print("accuracy: ", accuracy_score(y_test, preds))
print("precision:", precision_score(y_test, preds))
print("recall:   ", recall_score(y_test, preds))

log loss:  0.060941270092105745
accuracy:  0.9286307916060669
precision: 0.0
recall:    0.0


C:\Users\Ray Onsongo\anaconda3\envs\ray-env\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Interpret your model performance. How would it perform on different kinds of tasks? How much better is it than a "dummy" model that always chooses the majority class, or the logistic regression described at the start of the lab?

In [ ]:
# Replace None with appropriate text
"""
None
"""

## Conclusion

In this lab, you practiced the end-to-end machine learning process with multiple model algorithms, including tuning the hyperparameters for those different algorithms. You saw how nonparametric models can be more flexible than linear models, potentially leading to overfitting but also potentially reducing underfitting by being able to learn non-linear relationships between variables. You also likely saw how there can be a tradeoff between speed and performance, with good metrics correlating with slow speeds.